## Chatbot

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import pandas as pd
import numpy as np
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

# Download NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [WinError 10065] A
[nltk_data]     socket operation was attempted to an unreachable host>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [ ]:
import json
import pandas as pd

# Load the dataset from a JSON file
with open('dataset2.json') as f:
    data = json.load(f)

# Extract input, responses, and tags
tags = []
inputs = []
responses = {}
for intent in data['intents']:  # Access the 'intents' list within the JSON
    responses[intent['tag']] = intent['responses']
    for line in intent['patterns']:
        inputs.append(line)
        tags.append(intent['tag'])

# Create a DataFrame
data_df = pd.DataFrame({"inputs": inputs, "tags": tags})


In [4]:
# Remove punctuation and convert to lowercase
data_df['inputs'] = data_df['inputs'].apply(lambda wrd: ''.join([ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation]))


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder


# Tokenize the data
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data_df['inputs'])
train_sequences = tokenizer.texts_to_sequences(data_df['inputs'])

# Apply padding
x_train = pad_sequences(train_sequences)

# Encode the outputs
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(data_df['tags'])

# Define input shape
input_shape = x_train.shape[1]
print("Input shape:", input_shape)

# Define vocabulary size
vocabulary_size = len(tokenizer.word_index)
print("Number of unique words:", vocabulary_size)

# Define output length
output_length = len(label_encoder.classes_)
print("Output length:", output_length)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size + 1, 10, input_length=input_shape),
    tf.keras.layers.LSTM(10, return_sequences=True),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(output_length, activation='softmax')
])


Input shape: 14
Number of unique words: 242
Output length: 62


In [5]:

# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
num_epochs = 200
history = model.fit(x_train, y_train, epochs=num_epochs)

# Define function to predict answer
def predict_answer(model, tokenizer, label_encoder, question):
    question = ''.join([letters.lower() for letters in question if letters not in string.punctuation])
    texts_p = [question]
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)
    output = model.predict(prediction_input)
    output = output.argmax()
    response_tag = label_encoder.inverse_transform([output])[0]
    return response_tag


Epoch 1/200
9/9 [==============================] - 4s 13ms/step - loss: 4.1285 - accuracy: 0.0038
Epoch 2/200
9/9 [==============================] - 0s 11ms/step - loss: 4.1237 - accuracy: 0.0305
Epoch 3/200
9/9 [==============================] - 0s 29ms/step - loss: 4.1196 - accuracy: 0.0687
Epoch 4/200
9/9 [==============================] - 0s 12ms/step - loss: 4.1152 - accuracy: 0.0649
Epoch 5/200
9/9 [==============================] - 0s 12ms/step - loss: 4.1096 - accuracy: 0.0611
Epoch 6/200
9/9 [==============================] - 0s 15ms/step - loss: 4.1021 - accuracy: 0.0649
Epoch 7/200
9/9 [==============================] - 0s 13ms/step - loss: 4.0914 - accuracy: 0.0649
Epoch 8/200
9/9 [==============================] - 0s 27ms/step - loss: 4.0773 - accuracy: 0.0611
Epoch 9/200
9/9 [==============================] - 0s 13ms/step - loss: 4.0571 - accuracy: 0.0611
Epoch 10/200
9/9 [==============================] - 0s 11ms/step - loss: 4.0346 - accuracy: 0.0534
Epoch 11/200
9/9 [=

In [6]:
import random 
# Start chatbot
while True:
    user_input = input('You: ')
    if user_input.lower() == 'goodbye':
        print("Chatbot: Goodbye!")
        break
    response_tag = predict_answer(model, tokenizer, label_encoder, user_input)
    print("Chatbot:", random.choice(responses.get(response_tag, ["I'm sorry, I don't understand that."])))

1/1 [==============================] - 1s 858ms/step
Chatbot: Anda bisa mencoba minum obat pereda nyeri atau menggunakan bantalan hangat untuk meredakan sakit perut.
1/1 [==============================] - 0s 42ms/step
Chatbot: Beberapa makanan yang dapat membantu meredakan nyeri menstruasi adalah: 

1. Makanan yang kaya akan serat seperti buah-buahan, sayuran, dan biji-bijian. 
2. Makanan yang mengandung magnesium seperti bayam, kacang-kacangan, dan biji-bijian. 
3. Makanan yang mengandung vitamin E seperti alpukat, kacang-kacangan, dan minyak zaitun [^5^][5].
1/1 [==============================] - 0s 48ms/step
Chatbot: Beberapa gejala umum menstruasi meliputi sakit perut, mual, sakit kepala, dan perubahan suasana hati. Ini adalah hal yang normal selama menstruasi.
1/1 [==============================] - 0s 48ms/step
Chatbot: Pembalut wanita dan tampon adalah dua jenis produk yang digunakan untuk menyerap darah menstruasi. Berikut adalah beberapa jenis pembalut wanita yang umum digunaka